In [1]:
# Importing Libraries
import pandas as pd
import numpy as np
import os
from os.path import join

In [53]:
ORIGINAL_DATA_PATH = "../Original Data"

In [54]:
orders_df = pd.read_csv(join(ORIGINAL_DATA_PATH, "orders.csv"))

In [55]:
prods_df_clean_no_dups = pd.read_csv(join(ORIGINAL_DATA_PATH, "products.csv"))

In [56]:
prods_df = pd.read_csv(join(ORIGINAL_DATA_PATH, "products.csv"))

## Step 1: Consistency Checks omitted 'prods_df'

In [57]:
# cleaned all duplicates
prods_df_clean_no_dups.shape

(49693, 5)

In [58]:
prods_df.describe

<bound method NDFrame.describe of        product_id                                       product_name  \
0               1                         Chocolate Sandwich Cookies   
1               2                                   All-Seasons Salt   
2               3               Robust Golden Unsweetened Oolong Tea   
3               4  Smart Ones Classic Favorites Mini Rigatoni Wit...   
4               5                          Green Chile Anytime Sauce   
...           ...                                                ...   
49688       49684          Vodka, Triple Distilled, Twist of Vanilla   
49689       49685                 En Croute Roast Hazelnut Cranberry   
49690       49686                                   Artisan Baguette   
49691       49687         Smartblend Healthy Metabolism Dry Cat Food   
49692       49688                             Fresh Foaming Cleanser   

       aisle_id  department_id  prices  
0            61             19     5.8  
1           104    

In [59]:
prods_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49693 entries, 0 to 49692
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_id     49693 non-null  int64  
 1   product_name   49677 non-null  object 
 2   aisle_id       49693 non-null  int64  
 3   department_id  49693 non-null  int64  
 4   prices         49693 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 1.9+ MB


## Step 2: Run describe on orders

In [60]:
orders_df.describe()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


## Summary of 'df.describe() output for 'orders_df'
**All numeric columns are within expected ranges with no apparent outlier or incorrect values.
**'order_dow' ranges from 0 to 6 giving it a total of excatly 7 values for the 7 days of the week.
**'order_dow' column doesn't contain any strange or incorrect values.
**the 50th percentile is 3 in this case,making it the exact center of 0 and 6.
**'order_hour_of_day' ranges from 0 to 23 giving it a total of 24 hours in a day
**'days_since_prior_order' has -206,209 missing values which matches the number of unique users.This is expected and likely represents first-time orders(no prior data to compute days form).

In [61]:
## Step 3: Check for Mixed-Type Data in 'orders_df'
orders_df.dtypes

order_id                    int64
user_id                     int64
eval_set                   object
order_number                int64
order_dow                   int64
order_hour_of_day           int64
days_since_prior_order    float64
dtype: object

In [62]:
# Check for mixed types
for col in orders_df.columns.tolist():
 weird = (orders_df[[col]].map(type) != orders_df[[col]].iloc[0].apply(type)).any(axis = 1)
 if len (orders_df[weird]) > 0 :
     print(col)

**No mixed type columns were found. All columns contain consistent data types which enures stability during future processing steps.

## Step 4: Check Missing Values

In [71]:
# Finding missing values
orders_df.isnull().sum()

order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

**Only days_since_prior_order has missing values and that is 206,209. This number aligns perfectly with the number of unique users which strongly suggest that these missing values represents first time orders where there's no prior order to compute the number of days from.

In [79]:
# Handling missing values
orders_df['days_since_prior_order'] = orders_df['days_since_prior_order'].fillna(1)

**This approach keeps the column nummerical and avoids introducing outliers. Chose 1 instead of 0 to prevent confusion with same day orders and ensure smooth analysis along side the line.

## Step 5: Check for Duplicates

In [64]:
# cheking for duplictaes
orders_df.shape

(3421083, 7)

In [68]:
orders_df.duplicated().sum()

np.int64(0)

In [67]:
orders_df[orders_df.duplicated()]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order


** All entries in the 'orders_df' are unique and no action is required because 0 duplicate rows were found.

**The combination of these 2 methods allows you to identify and confirm whether duplicates exist. one shows the actual duplicated rows and the other gives the count. It's a standard and effective approach for checking datasets intergrity before deeper analysis.

## Step 6: Export data

In [69]:
# Checking for path
path = os.getcwd()

In [70]:
# Exporting files
prepared_data_path = os.path.join(path, "Prepared Data")
os.makedirs(prepared_data_path, exist_ok=True)

In [81]:
orders_df.to_csv(os.path.join(prepared_data_path, "orders_clean.csv"))

In [82]:
prods_df_clean_no_dups.to_csv(os.path.join(prepared_data_path, "products.csv"))
                              